# Weather Prediction Model

### Importing all Libraries

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import warnings
import sklearn
warnings.simplefilter('ignore', FutureWarning)
pd.options.mode.chained_assignment = None

### Extracting data from API

In [2]:
data = pd.DataFrame()
offset = 0
while True:
    response = requests.get(f'https://data.edmonton.ca/resource/s4ws-tdws.json?$offset={offset}')
    new_data = pd.DataFrame(json.loads(response.text))
    print(f'Fetched {len(new_data)} new rows')
    if len(new_data) == 0:
        break
    data = pd.concat([data, new_data])
    offset += len(new_data)
data.head()

Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 1000 new rows
Fetched 10

,row_id,station_id,station_name,station_province,station_latitude,station_longitude,location,station_elevation_m,station_climate_identifier,station_wmo_identifier,...,:@computed_region_bhid_jnzh,:@computed_region_ecxu_fw7u,snow_on_ground_cm,speed_of_maximum_wind_gust_km_h,direction_of_maximum_wind_gust_10s_deg,data_quality,total_rain_flag,total_snow_flag,total_rain_mm,total_snow_cm
0,2721420210414,27214,EDMONTON BLATCHFORD,ALBERTA,53.57,-113.52,"{'latitude': '53.57', 'longitude': '-113.52', ...",671.00,3012209,71157,...,114,221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2721420210415,27214,EDMONTON BLATCHFORD,ALBERTA,53.57,-113.52,"{'latitude': '53.57', 'longitude': '-113.52', ...",671.00,3012209,71157,...,114,221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2721420210510,27214,EDMONTON BLATCHFORD,ALBERTA,53.57,-113.52,"{'latitude': '53.57', 'longitude': '-113.52', ...",671.00,3012209,71157,...,114,221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2721420210511,27214,EDMONTON BLATCHFORD,ALBERTA,53.57,-113.52,"{'latitude': '53.57', 'longitude': '-113.52', ...",671.00,3012209,71157,...,114,221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2721420210127,27214,EDMONTON BLATCHFORD,ALBERTA,53.57,-113.52,"{'latitude': '53.57', 'longitude': '-113.52', ...",671.00,3012209,71157,...,114,221,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Creating a copy

In [3]:
dataCopy = data.copy()

In [28]:
data = dataCopy

### Data Preprocessing

In [29]:
data.columns

Index(['row_id', 'station_id', 'station_name', 'station_province',
       'station_latitude', 'station_longitude', 'location',
       'station_elevation_m', 'station_climate_identifier',
       'station_wmo_identifier', 'station_tc_identifier', 'year', 'month',
       'day', 'date', 'maximum_temperature_c', 'minimum_temperature_c',
       'mean_temperature_c', 'heating_degree_days_c', 'cooling_degree_days_c',
       'total_precipitation_mm', ':@computed_region_da6r_6gkw',
       ':@computed_region_uxfu_uxx8', ':@computed_region_ive2_3q52',
       ':@computed_region_bhid_jnzh', ':@computed_region_ecxu_fw7u',
       'snow_on_ground_cm', 'speed_of_maximum_wind_gust_km_h',
       'direction_of_maximum_wind_gust_10s_deg', 'data_quality',
       'total_rain_flag', 'total_snow_flag', 'total_rain_mm', 'total_snow_cm'],
      dtype='object')

In [30]:
data.station_name.value_counts()

Edmonton International Airport    19403
Stony Plain                       17655
Downtown                          16606
Namao                             16378
New Sarepta                        6197
Villeneuve Airport                 4079
Name: station_name, dtype: int64

#### For further details of the following replacement refer to the link:
https://www.mapcustomizer.com/map/Weather%20Analysis <br>
https://www.windfinder.com/#10/53.5578/-113.4544/spot

In [31]:
data.replace({'station_name': {'EDMONTON STONY PLAIN CS': 'Stony Plain', 
                               'EDMONTON BLATCHFORD': 'Downtown', 
                               'EDMONTON INTERNATIONAL CS': 'Edmonton International Airport', 
                               'EDMONTON STONY PLAIN': 'Stony Plain', 
                               'EDMONTON NAMAO AWOS A': 'Namao', 
                               'OLIVER AGDM': 'Namao', 
                               'THORSBY AGCM': 'Edmonton International Airport', 
                               'NEW SAREPTA AGCM': "New Sarepta", 
                               'EDMONTON INTL A': 'Edmonton International Airport',
                               'EDMONTON VILLENEUVE A': 'Villeneuve Airport', 
                               'EDMONTON NAMAO A': 'Namao',
                               'LEGAL AGCM': 'Downtown',
                               'EDMONTON SOUTH CAMPUS': "Downtown"}}, inplace=True)
data.station_name.value_counts()

Edmonton International Airport    19403
Stony Plain                       17655
Downtown                          16606
Namao                             16378
New Sarepta                        6197
Villeneuve Airport                 4079
Name: station_name, dtype: int64

In [32]:
data_info = data[['station_name', 'station_province', 'station_latitude',
                   'station_longitude', 'station_elevation_m']].dropna()
data_info[['station_latitude', 'station_longitude', 'station_elevation_m']] = data_info[['station_latitude', 'station_longitude', 'station_elevation_m']].astype(float)
data_info = data_info.drop_duplicates(subset=['station_name', 'station_province'])
data_info.groupby(['station_name', 'station_province', 'station_latitude',
                   'station_longitude', 'station_elevation_m']).size()

station_name                    station_province  station_latitude  station_longitude  station_elevation_m
Downtown                        ALBERTA           53.57             -113.52            671.0                  1
Edmonton International Airport  ALBERTA           53.22             -113.89            754.0                  1
Namao                           ALBERTA           53.67             -113.47            687.9                  1
New Sarepta                     ALBERTA           53.26             -113.17            765.0                  1
Stony Plain                     ALBERTA           53.55             -114.11            766.3                  1
Villeneuve Airport              ALBERTA           53.67             -113.86            687.6                  1
dtype: int64

In [33]:
data.columns

Index(['row_id', 'station_id', 'station_name', 'station_province',
       'station_latitude', 'station_longitude', 'location',
       'station_elevation_m', 'station_climate_identifier',
       'station_wmo_identifier', 'station_tc_identifier', 'year', 'month',
       'day', 'date', 'maximum_temperature_c', 'minimum_temperature_c',
       'mean_temperature_c', 'heating_degree_days_c', 'cooling_degree_days_c',
       'total_precipitation_mm', ':@computed_region_da6r_6gkw',
       ':@computed_region_uxfu_uxx8', ':@computed_region_ive2_3q52',
       ':@computed_region_bhid_jnzh', ':@computed_region_ecxu_fw7u',
       'snow_on_ground_cm', 'speed_of_maximum_wind_gust_km_h',
       'direction_of_maximum_wind_gust_10s_deg', 'data_quality',
       'total_rain_flag', 'total_snow_flag', 'total_rain_mm', 'total_snow_cm'],
      dtype='object')

In [34]:
cols_to_keep = ['station_name', 'year', 'month', 'day', 'maximum_temperature_c', 'minimum_temperature_c',
                'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']
data = data[cols_to_keep]
data

,station_name,year,month,day,maximum_temperature_c,minimum_temperature_c,speed_of_maximum_wind_gust_km_h,total_rain_mm,total_snow_cm
0,Downtown,2021,4,14,16.2,-0.7,NaN,NaN,NaN
1,Downtown,2021,4,15,17.5,-1.2,NaN,NaN,NaN
2,Downtown,2021,5,10,16.2,1.4,NaN,NaN,NaN
3,Downtown,2021,5,11,17.3,6.2,NaN,NaN,NaN
4,Downtown,2021,1,27,-15.1,-17.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
313,Edmonton International Airport,2023,12,30,3.7,-5.6,42,0.0,0.0
314,Edmonton International Airport,2023,12,30,4.0,-5.9,38,NaN,NaN
315,Villeneuve Airport,2023,12,30,3.4,-8.4,NaN,NaN,NaN
316,Stony Plain,2023,12,29,10.0,-4.0,NaN,0.0,0.0


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80318 entries, 0 to 317
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   station_name                     80318 non-null  object
 1   year                             80318 non-null  object
 2   month                            80318 non-null  object
 3   day                              80318 non-null  object
 4   maximum_temperature_c            77013 non-null  object
 5   minimum_temperature_c            77527 non-null  object
 6   speed_of_maximum_wind_gust_km_h  26367 non-null  object
 7   total_rain_mm                    24026 non-null  object
 8   total_snow_cm                    23779 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [36]:
# Fill missing values with 0
data[['speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']] = data[['speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']].fillna(0)
data.dropna(inplace=True)

# Replace '<31' with 31
data.replace({'speed_of_maximum_wind_gust_km_h': {'<31': 31}}, inplace=True)

# Convert specified columns to integer
int_columns = ['year', 'month', 'day']
data.loc[:, int_columns] = data[int_columns].astype(int)

# Convert specified columns to float
float_columns = ['maximum_temperature_c', 'minimum_temperature_c', 'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']
data.loc[:, float_columns] = data[float_columns].astype(float)

# Display information about DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76784 entries, 0 to 317
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   station_name                     76784 non-null  object
 1   year                             76784 non-null  object
 2   month                            76784 non-null  object
 3   day                              76784 non-null  object
 4   maximum_temperature_c            76784 non-null  object
 5   minimum_temperature_c            76784 non-null  object
 6   speed_of_maximum_wind_gust_km_h  76784 non-null  object
 7   total_rain_mm                    76784 non-null  object
 8   total_snow_cm                    76784 non-null  object
dtypes: object(9)
memory usage: 5.9+ MB


In [38]:
data.sort_values(by=['year', 'month', 'day', 'station_name'], inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis=1, inplace=True)

In [39]:
data.to_csv('weather_data.csv', index=False)

### Splitting the data

In [40]:
X, Y = data[['station_name', 'year', 'month', 'day']], data[['maximum_temperature_c', 'minimum_temperature_c', 'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']]

### Training Data range: 2000-01-01 to 2020-12-31
### Validate Data Range: 2021-01-01 to 2021-12-31
### Testing Data range: 2022-01-01 to end
train_data = data[(data['year'] >= 2000) & (data['year'] <= 2020)]
validate_data = data[(data['year'] == 2021)]
test_data = data[(data['year'] >= 2022)]

X_train, Y_train = train_data[['station_name', 'year', 'month', 'day']], train_data[['maximum_temperature_c', 'minimum_temperature_c', 'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']]
X_validate, Y_validate = validate_data[['station_name','year', 'month', 'day']], validate_data[['maximum_temperature_c', 'minimum_temperature_c', 'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']]
X_test, Y_test = test_data[['station_name', 'year', 'month', 'day']], test_data[['maximum_temperature_c', 'minimum_temperature_c', 'speed_of_maximum_wind_gust_km_h', 'total_rain_mm', 'total_snow_cm']]

### Model Building

In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['station_name'])
    ], remainder='passthrough')

Mean Squared Error: 173.20101504691092


In [48]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error

models = [RandomForestRegressor(), 
          HistGradientBoostingRegressor(), 
          XGBRegressor(), 
          Ridge(), 
          KNeighborsRegressor(n_neighbors=6, weights='distance'), 
          DecisionTreeRegressor()]

trainedModels = {}
count = 1
for model in models:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', MultiOutputRegressor(model))])
    pipeline.fit(X_train, Y_train)
    Y_validate_pred = pipeline.predict(X_validate)
    print(f'{model} MSE:', mean_squared_error(Y_validate, Y_validate_pred))
    trainedModels["model_%s" % count] = pipeline
    count += 1

RandomForestRegressor() MSE: 170.3871148648198
HistGradientBoostingRegressor() MSE: 127.07971702582003
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...) MSE: 148.84134041881555
Ridge() MSE: 173.1650173918037
KNeighborsRegressor(n_neighbors=6, weights='distance') MSE: 148.4625602250294
DecisionTreeRegre

### Selecting a model and fine tuning it

In [57]:
### Select the best model
bestModel = trainedModels["model_2"]

### Fine tuning it by using KFold and RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

param_grid = {
    'regressor__estimator__max_iter': [100, 200, 300, 400, 500],
    'regressor__estimator__max_depth': [10, 20, 30, 40, 50, None],
    'regressor__estimator__max_leaf_nodes': [10, 20, 30, 40, 50],
    'regressor__estimator__l2_regularization': [0.1, 0.2, 0.3, 0.4, 0.5],
    'regressor__estimator__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3]
}

kf = KFold(n_splits=5, shuffle=False)
random_search = RandomizedSearchCV(bestModel, param_distributions=param_grid, 
                                   n_iter=100, cv=kf, n_jobs=-1, verbose=3, random_state=42)

random_search.fit(X_train, Y_train)

print("Best parameters found: ", random_search.best_params_)
print("Lowest MSE found: ", random_search.best_score_)
print("Best model: ", random_search.best_estimator_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found:  {'regressor__estimator__max_leaf_nodes': 10, 'regressor__estimator__max_iter': 500, 'regressor__estimator__max_depth': 50, 'regressor__estimator__learning_rate': 0.01, 'regressor__estimator__l2_regularization': 0.1}
Lowest MSE found:  0.3385309649922195
Best model:  Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['station_name'])])),
                ('regressor',
                 MultiOutputRegressor(estimator=HistGradientBoostingRegressor(l2_regularization=0.1,
                                                                              learning_rate=0.01,
                                                                              max_depth=50,
                                                                              max_i

In [58]:
bestModel = random_search.best_estimator_

In [59]:
Y_validate_pred = bestModel.predict(X_validate)
print('MSE: ', mean_squared_error(Y_validate, Y_validate_pred))

MSE:  111.82288655941667
